# Distributed Parallelism using MPI

MPI, the Message Passing Interface is a de-facto industry standard for massively parallel computing. Its origins go back to the 1990s and over the course of its development it has been shown to scale well to thousands of compute nodes.

MPI is not the only way to do distributed parallelism in Julia. In fact Julia ships by itself with the 
[`Distributed`](https://docs.julialang.org/en/v1/stdlib/Distributed/) standard library, which is preferred by some users over MPI.jl due to its more convenient and more understandable programming model. Moreover `Distributed` integrates better into an interactive programming workflow.

Albeit these advantages the fact that (a) HPC users are often very experienced in MPI programming and (b) compute clusters often provide custom MPI implementations making implicit use of e.g. hardware specialities or the details of the communication topology.

## MPI libraries and Julia

* [MPI](https://www.mpi-forum.org/) is a community standard, which specifies how an MPI library should be have. In practice there are several popular implementations, such as OpenMPI, IntelMPI, MPICH, ...
* [MPI.jl](https://github.com/JuliaParallel/MPI.jl) is Julia's package and interface to an MPI implementation.

* Installing MPI.jl (`] add MPI`) automatically takes care of downloading and installing *an* MPI implementation, which is compatible with the current machine, which is usually sufficient for laptop or desktop machines.
* If a special MPI library is desired one can install any MPI library manually and direct  MPI.jl to the manual installation via `ENV["JULIA_MPI_BINARY"]="system"`.
* On HPC clusters with Julia support, the typical MPI modules are often already set up in a way, such that MPI.jl directly employs the correct, cluster-specific configuration. If this is not the case for your compute cluster, try using `ENV["JULIA_MPI_BINARY"]="system"` [and related environment variables](https://juliaparallel.org/MPI.jl/stable/configuration/#environment_variables) for configuration.

## mpiexecjl

* As we will discuss in a second to run an MPI-aware program in parallel one needs to use the `mpiexec` (alias `mpirun`) executable.
* The standard does not allow to mix versions between the MPI library used inside the code and this wrapper executable.
* Julia therefore provides th [`mpiexecjl`](https://juliaparallel.org/MPI.jl/stable/configuration/) wrapper to ensure the `mpiexec` version agrees with the MPI library used inside Julia. To install this wrapper run:

In [ ]:
using MPI
MPI.install_mpiexecjl()

On Linux this installs the wrapper to `$HOME/.julia/bin`. Please make a note of the path this displays on your computer as you will need to use this wrapper script for starting MPI-parallelised Julia programs.

## MPI crash course

Due to its importance for parallel computing nummerous standard resources for MPI programming can be found. In principle all their ideas and code examples can be directly employed for learning MPI-based parallel programming in Julia. I will only hint at the most basic MPI functionality here, to give you a taste how code examples for C and FORTRAN would need to be adapted.

### Programming model

The basic MPI programming model is called *single program, multiple data*:
- At startup of a program the MPI runtime tools `mpiexec` (alias  `mpirun`) take care to start the processes.
- **Each process executes the same program**
- Differing behaviour between processes must be implemented manually, e.g. using rank information and conditionals
- As a result: Individual **processes follow different paths** through the program, can thus get out of sync.
- Constructs for **inter-process communication** allow to synchronise, share, gather or broadcast data.

#### Hello world example

Consider the example `22_MPI_examples/1_mpi_hello.jl`:
```julia
using MPI    

# Has to be called before doing anything else with MPI    
MPI.Init()    

comm = MPI.COMM_WORLD    
rank = MPI.Comm_rank(comm)    
world_size = MPI.Comm_size(comm)    
    
println("Hello from rank $(rank) of $(world_size)")    
```

#### Running the example

To run this example with `4` processors in parallel, we execute:

```
path/to/mpiexecjl -np 4 julia --project 1_mpi_hello.jl
```

where `path/to/mpiexecjl` is the path to the `mpiexecjl` wrapper script displayed above.

### Fundamental MPI functions

We first discuss the fundamental MPI functions in this example.

* `MPI.Init()`: Sets up the MPI runtime. Has to be called before doing anything else `MPI`-related. Usually called at the top of your code. 
* `MPI.Finalize()`: Finishes the MPI operations. After this no more `MPI`-related functions can be used. Julia calls this function automatically if not yet done by the user.
* `MPI.COMM_WORLD`: Default communicator consisting of all processes created when starting the program (so in the above example `4` processes). 
* `MPI.Comm_rank(comm)`: Rank (integer identifier) of the calling process
* `MPI.Comm_size(comm)`: Total number of processes in the communicator `comm`.

### Point-to-point Communication

Point-to-point blocking communication is achieved using `MPI.Send` and `MPI.Recv`, see the example `22_MPI_examples/2_mpi_basic_communication.jl`. The syntax is as follows:
* `MPI.Send(data, destination, tag, communicator)` and `MPI.Recv!(data, origin, tag, communicator)`:
  * `data`: E.g. array containing some values
  * `destination` / `origin`: Rank of the target / source process
  * `tag`: Integer identifying the message (for our simple examples just set to `0`)

* Since this communication is blocking the user has to ensure that every data which is sent will be retrieved by the desired destination process. Otherwise deadlocks can occurr.
* Note that there are [other point-to-point communication variants](https://juliaparallel.org/MPI.jl/stable/reference/pointtopoint/), which overcome this limitation.

### Collective communication

Collective communication involves all ranks of a communicator.

* **Synchronisation**:
    * `MPI.Barrier(communicator)`
    * See `22_MPI_examples/3_mpi_barrier.jl`
* **Broadcast** (one-to-all):
    * `data = MPI.bcast(data, origin, communicator)`.
    * Sends data from `origin` to all processes in the `communicator`. 
    * `data` can be a buffer or similar
    * See `22_MPI_examples/4_mpi_broadcast.jl`
* **Reduction** (accumulate in one):
    * `result = MPI.Reduce(data, op, destination, communicator)`
    * `op`: Reduction function, e.g. `+`
    * Gathers at `destination` the result by applying the reduction function to the data scattered across the ranks of the `communicator`.
    * See `22_MPI_examples/5_mpi_reduction.jl`

### Conveniences of MPI.jl

* If possible MPI.jl has a consistent naming convention from C to Julia:
   * `MPI_*` in C -> `MPI.*` in Julia
   * E.g. `MPI_COMM_WORLD` -> `MPI.COMM_WORLD`, `MPI_Comm_size` -> `MPI.Comm_size`
* It is often possible to reduce the number of function arguments in MPI.jl over their counterparts in the MPI C libraries, since Julia objects (such as arrays) keep metadata about sizes, types etc.
* MPI.jl functions can often automatically handle data of **built-in and custom Julia types** (i.e. custom `struct`s)
* The same applies for functions passed to `MPI.Reduce` and similar functions.
* See `22_MPI_examples/6_mpi_custom_types.jl`.